In [276]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.stats import skew

In [292]:
train = pd.read_csv('train.csv').set_index('Id')
test = pd.read_csv('test.csv').set_index('Id')

In [293]:
all_data = pd.concat((train.loc[:, 'MSSubClass':'SaleCondition'], test.loc[:, 'MSSubClass':'SaleCondition']))

In [279]:
train['SalePrice'] = np.log1p(train.SalePrice)
test['SalePrice'] = np.log1p(train.SalePrice)

In [294]:
num_cat = all_data.select_dtypes(include=[np.number])

In [295]:
for columns in num_cat:
    if num_cat[columns].dropna().skew() > 0.75:
        num_cat.loc[:,columns] = np.log1p(num_cat[columns])

/home/aniketmaurya/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [296]:
num_cat = num_cat.fillna(num_cat.mean())

In [297]:
df_train = num_cat.iloc[:train.shape[0], :]
df_test = num_cat.iloc[train.shape[0]:,:]
Y = train['SalePrice']

In [370]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_train, Y, train_size=0.9)

/home/aniketmaurya/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [392]:
from sklearn.ensemble import GradientBoostingRegressor

In [422]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()

In [423]:
model.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [424]:
model.score(X_train, Y_train), model.score(X_test, Y_test)

(0.9627095930371335, 0.8668220773651748)

In [425]:
res = pd.DataFrame(model.predict(df_test), columns = ['SalePrice']).set_index(df_test.index)

In [426]:
res.to_csv('housepricing')

In [427]:
model.score(df_train, Y)

0.9489290737113952